# Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Read in Data

In [2]:
# Set working directory
%cd C:/Users/sriha/OneDrive/Desktop/DS/My work/life-expectancy
    
# Unpickle engineered train data
X_train = pd.read_pickle('Data/X_train_engineered.pkl')

# Unpickle engineered test data
X_test = pd.read_pickle('Data/X_test_engineered.pkl')

C:\Users\sriha\OneDrive\Desktop\DS\My work\life-expectancy


In [3]:
display(X_train.head())
display(X_test.head())

,Country,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Status_Developing
647,Croatia,2009,2.995732,2.302585,12.21,7.678502,4.584967,1.098612,2.397895,4.574711,8.18,4.574711,0.09531,9.558045,13.001278,0.993252,0.955511,0.589452,14.20,0
1174,Iceland,2011,3.951244,2.197225,8.13,2.085435,4.543295,0.000000,2.351375,4.564348,8.65,4.564348,0.09531,3.854754,10.370831,0.641854,0.641854,0.638691,18.60,0
234,Belarus,2005,5.533389,0.693147,11.01,3.842675,4.605170,0.693147,0.693147,4.595120,6.89,4.605170,0.09531,8.048059,16.083910,1.193922,1.252763,0.538246,14.40,1
2789,United Republic of Tanzania,2004,6.120297,4.615121,3.81,0.000000,4.564348,7.258412,5.043425,4.564348,4.10,4.564348,2.46810,7.216045,14.029974,2.174752,2.174752,0.500169,12.05,1
2281,Seychelles,2015,5.129899,2.079442,7.41,0.000000,4.595120,0.000000,2.197225,4.584967,3.73,4.584967,0.09531,7.339839,11.444861,1.902108,1.945910,0.577175,14.10,1


,Country,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Status_Developing
1610,Maldives,2007,4.418841,2.251292,1.78,2.013768,4.595120,3.044522,2.397895,4.595120,2.033398,4.595120,0.095310,3.986431,5.857933,2.727853,2.734368,0.489806,11.9,1
2622,Togo,2010,5.780744,2.708050,1.21,3.760303,4.430817,4.795791,3.091042,4.430817,1.851599,4.430817,1.568616,6.192207,13.389260,2.116256,2.104134,0.370874,11.0,1
642,Croatia,2014,4.584967,2.079442,12.14,7.541736,4.564348,0.000000,2.251292,4.564348,2.174752,4.564348,0.095310,9.508107,15.259694,0.916291,0.916291,0.598837,15.1,0
1192,India,2009,1.098612,7.170888,2.50,0.612038,3.637586,10.936476,7.438972,4.304065,1.682688,4.317488,0.182322,3.011496,18.614825,3.332205,3.360375,0.446607,10.5,1
2553,Tajikistan,2015,5.087596,2.397895,0.35,0.000000,4.574711,1.386294,2.484907,4.574711,1.902108,4.574711,0.182322,6.824023,15.961284,1.526056,1.547563,0.485508,11.3,1


# Create ModelContainer Object

In [4]:
class ModelContainer():
    
    def __init__(self):
        self.models = {}
        self.mean_mse = {}
        self.std_mse = {}
        self.best_model = None
        self.kfolds = 0

        
    def add_model(self, model):
        self.models[model[0]] = model[1]

        
    def get_results(self, X, y, kfolds=5):
        for model_name, model in self.models.items():
          neg_mse = cross_val_score(model, X, y, cv=kfolds, scoring='neg_mean_squared_error')
          self.mean_mse[model_name] = -1.0*np.mean(neg_mse)
          self.std_mse[model_name] = np.std(neg_mse)
          self.kfolds = kfolds
          print(model_name, 'cross-validated.')
 

    def print_summary(self):
        print('\nModel Summaries:\n')
        for model_name, model in self.models.items():
            print(model_name, ':')
            print('Mean MSE over', self.kfolds.n_splits, 'folds:',  self.mean_mse[model_name], '+/-', self.std_mse[model_name],'\n')